In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Use Gemini 2.5 Pro to create MCP Server



<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/wadave/vertex_ai_mcp_samples/blob/main/create_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  
  
  
  <td style="text-align: center">
    <a href="https://github.com/wadave/vertex_ai_mcp_samples/blob/main/create_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Dave Wang](https://github.com/wadave) |

## Overview

This notebook demonstrates how to use Gemini 2.5 pro create MCP server codes.

### Prerequisites
- Get google pai key
- Set up google api key and save it in google cloud secret manager (optional)

## Get started

### Install Google Gen AI SDK and other required packages


In [ ]:
%pip install --upgrade --quiet google-genai google-cloud-secret-manager mcp geopy black google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

from google import genai

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

#client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [2]:
from google.cloud import secretmanager
from google import genai
import datetime

from google.genai.types import (
    CreateBatchJobConfig,
    CreateCachedContentConfig,
    EmbedContentConfig,
    FunctionDeclaration,
    GenerateContentConfig,
    Part,
    SafetySetting,
    Tool,
)

import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from typing import Any, List
import asyncio
from google.genai import types
from typing import List
from util import(
    access_secret_version,
    get_url_content,
    format_python
)

### Option 1 Get API key from secret manager

In [3]:
# TODO you need to create a 'google_api_key' and save it in secret manager
api_key = access_secret_version(PROJECT_ID, secret_id="google_api_key", version_id="latest")

### Option 2 Get api key from environment


In [ ]:
# api_key=os.getenv("GEMINI_API_KEY")

In [4]:
# TODO(developer): Update below line
API_KEY = api_key

client = genai.Client(api_key=API_KEY)

MODEL25_ID = "gemini-2.5-pro-exp-03-25"

### Get system instruction context info

In [5]:
# The URL you want to fetch
url = 'https://modelcontextprotocol.io/quickstart/server'

In [6]:
reference_content = get_url_content(url)

Successfully fetched content


In [ ]:
from pydantic import BaseModel
class response_schema(BaseModel):
    python_code: str
    description: str
    
system_instruction = f"""
  You are an MCP server export.
  Your mission is to write python code for MCP server.
  Here's the MCP server development guide and example
  {reference_content}
  
"""


In [9]:
def generate_mcp_server(prompt): 
    response = client.models.generate_content(
        model=MODEL25_ID,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            response_mime_type="application/json",
            response_schema=response_schema,
        ),
    )
    
    return response.text

### Gemini Agent

In [8]:
# Define gemini agent loop
MODEL_ID = "gemini-2.0-flash-001"
async def agent_loop(prompt: str, client: genai.Client, session: ClientSession):
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Initialize the connection
    await session.initialize()
    
    # --- 1. Get Tools from Session and convert to Gemini Tool objects ---
    mcp_tools = await session.list_tools()
    tools = types.Tool(function_declarations=[
        {
            "name": tool.name,
            "description": tool.description,
            "parameters": tool.inputSchema,
        }
        for tool in mcp_tools.tools
    ])
    
    # --- 2. Initial Request with user prompt and function declarations ---
    response = await client.aio.models.generate_content(
        model=MODEL_ID,  # Or your preferred model supporting function calling
        contents=contents,
        config=types.GenerateContentConfig(
            temperature=0,
            tools=[tools],
        ),  # Example other config
    )
    
    # --- 3. Append initial response to contents ---
    contents.append(response.candidates[0].content)

    # --- 4. Tool Calling Loop ---            
    turn_count = 0
    max_tool_turns = 5
    while response.function_calls and turn_count < max_tool_turns:
        turn_count += 1
        tool_response_parts: List[types.Part] = []

        # --- 4.1 Process all function calls in order and return in this turn ---
        for fc_part in response.function_calls:
            tool_name = fc_part.name
            args = fc_part.args or {}  # Ensure args is a dict
            print(f"Attempting to call MCP tool: '{tool_name}' with args: {args}")

            tool_response: dict
            try:
                # Call the session's tool executor
                tool_result = await session.call_tool(tool_name, args)
                print(f"MCP tool '{tool_name}' executed successfully.")
                if tool_result.isError:
                    tool_response = {"error": tool_result.content[0].text}
                else:
                    tool_response = {"result": tool_result.content[0].text}
            except Exception as e:
                tool_response = {"error":  f"Tool execution failed: {type(e).__name__}: {e}"}
            
            # Prepare FunctionResponse Part
            tool_response_parts.append(
                types.Part.from_function_response(
                    name=tool_name, response=tool_response
                )
            )

        # --- 4.2 Add the tool response(s) to history ---
        contents.append(types.Content(role="user", parts=tool_response_parts))
        print(f"Added {len(tool_response_parts)} tool response parts to history.")

        # --- 4.3 Make the next call to the model with updated history ---
        print("Making subsequent API call with tool responses...")
        response = await client.aio.models.generate_content(
            model=MODEL_ID,
            contents=contents,  # Send updated history
            config=types.GenerateContentConfig(
                temperature=1.0,
                tools=[tools],
            ),  # Keep sending same config
        )
        contents.append(response.candidates[0].content)

    if turn_count >= max_tool_turns and response.function_calls:
        print(f"Maximum tool turns ({max_tool_turns}) reached. Exiting loop.")

    print("MCP tool calling loop finished. Returning final response.")
    # --- 5. Return Final Response ---
    return response
        


## Example 1:  Create Big Query MCP Server

In [10]:
prompt = """
  Please create an MCP server code for google cloud big query. It has two tools. One is to list tables for all datasets, the other is to describe a table. Google cloud project id and location will be provided for use. please use project id to access big query client.
  
  Return Python code within a JSON object formatted exactly as: {'python_code':    'your_generated_code', 'description':'your description'}
"""


In [11]:
response_text= generate_mcp_server(prompt)
python_code=json.loads(response_text)['python_code']

### Format python code

In [12]:
format_python(python_code, "bq_script2.py")

Successfully formatted the code and saved it to 'bq_script2.py'


In [14]:
bq_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your weather_server.py file
    args=["./bq_script2.py"],
)

In [15]:
async def run():
    async with stdio_client(bq_server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = "list my big query tables, project id is 'ace-chatbot-demo', location is 'us'"
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await agent_loop(prompt, client, session)
            return res
res = await run()
print(res.text)

Running agent loop with prompt: list my big query tables, project id is 'ace-chatbot-demo', location is 'us'
Attempting to call MCP tool: 'list_tables_for_all_datasets' with args: {'project_id': 'ace-chatbot-demo'}
MCP tool 'list_tables_for_all_datasets' executed successfully.
Added 1 tool response parts to history.
Making subsequent API call with tool responses...
MCP tool calling loop finished. Returning final response.
Here is a list of tables in project 'ace-chatbot-demo':

Dataset: aaa_demo_v0001_agents
  - insights_v3
  - interaction_logs

Dataset: aaa_demo_v0001_app
  - user_info

Dataset: conversation_sample
  - conversation_sample1
  - insights



### Sample 2: Medlineplus 
Create an MCP server for 
https://medlineplus.gov/about/developers/webservices/ API service

In [16]:
med_url ="https://medlineplus.gov/about/developers/webservices/"

In [17]:
med_api_details = get_url_content(med_url)

Successfully fetched content


In [18]:
prompt = f"""
  Please create an MCP server code for https://medlineplus.gov/about/developers/webservices/. It has one tool, get_medical_term. You provide a medical term, this tool will return explanation of the medial term
  
  Here's the API details:
  {med_api_details}
"""
def generate_mcp_server(prompt): 
    response = client.models.generate_content(
        model=MODEL25_ID,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            response_mime_type="application/json",
            response_schema=response_schema,
        ),
    )
    
    return response.text
response_text= generate_mcp_server(prompt)
python_code=json.loads(response_text)['python_code']

format_python(python_code, "med.py")

Successfully formatted the code and saved it to 'med.py'


In [19]:
med_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your weather_server.py file
    args=["./med.py"],
)

In [20]:
async def run():
    async with stdio_client(med_server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = "tell me detail of flu"
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await agent_loop(prompt, client, session)
            return res
res = await run()
print(res.text)

Running agent loop with prompt: tell me detail of flu
Attempting to call MCP tool: 'get_medical_term' with args: {'term': 'flu'}
MCP tool 'get_medical_term' executed successfully.
Added 1 tool response parts to history.
Making subsequent API call with tool responses...
MCP tool calling loop finished. Returning final response.
The flu, also called influenza, is a respiratory infection caused by viruses. Symptoms include fever, cough, sore throat, runny or stuffy nose, muscle or body aches, headaches, and fatigue. Some people may also have vomiting and diarrhea. It is best to get a flu vaccine every year to prevent it.

